In [43]:
%%writefile scripts/main.bash
#! /bin/bash

DATASET="femnist"
lr="0.001"
num_rounds="200"
clients_per_round="10"
base_line="fedco"

function run_fedco() {
    num_clusters="$1"
    python fl_main.py --dataset=${DATASET} --num_rounds=${num_rounds} --baseline=${base_line} --clients-per-round=${clients_per_round} --epochs=10 --test_batch=100 --batch-size=10 --lr=${lr} --eval_every=1 --k=${num_clusters} --seed_file=config/random_seed.csv --extra_params_file=../configs/fedrobust/job.yaml    
}

##################### Script #################################
cd /root/leaf/data/femnist
./preprocess.sh -s niid --sf 0.2 -k 30 -t sample --tf 0.8
cd /root/multi-center-fed-learning
git pull origin pfl-test
cd models
echo $(pwd)
echo "Running Training Pipeline......."
run_fedco 2
run_fedco 4
run_fedco 6

gsutil -m cp ./output/* gs://gs_bakcentral/json/



Overwriting scripts/main.bash


In [44]:
bucket_name = "gs_bakcentral"
s_name = "scripts/main.bash"
t_name = "src/main.bash"
upload_blob(bucket_name, s_name, t_name)

File scripts/main.bash uploaded to src/main.bash.


In [ ]:
cd /root/leaf/data/synthetic
rm -rf data
python main.py -num-tasks 1000 -num-classes 5 -num-dim 60
./preprocess.sh -s niid --sf 1.0 -k 5 -t sample --tf 0.6

In [ ]:
cd /root/leaf/data/femnist
./preprocess.sh -s niid --sf 0.3 -k 30 -t sample --tf 0.8


# copy following text to custom job training

In [20]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )


In [ ]:
/bin/bash
-c
gsutil cp gs://gs_bakcentral/src/* /root && cd /root/ && bash main.bash